Download the config files and other data from github

In [1]:
import os
if os.path.isfile('./../data/dmpipe_example.tar.gz'):
    !tar xzf ./../data/dmpipe_example.tar.gz
else:
    !curl -OL --output-dir ./../data/ https://raw.githubusercontent.com/fermiPy/fermipy-extras/master/data/dmpipe_example.tar.gz
    !tar xzf ./../data/dmpipe_example.tar.gz

In [2]:
from dmpipe.pipeline import Pipeline

Using /home/pmarinos/miniforge3/envs/fermipy/lib/python3.9/site-packages/dmsky/data for FileLibrary
Using /home/pmarinos/miniforge3/envs/fermipy/lib/python3.9/site-packages/dmsky/data/targets for TargetLibrary


First we build a pipeline object to analyze the dSphs, and point it at the config file for the dSphs analysis.

In [3]:
pipe = Pipeline(linkname = 'dSphs')

In [4]:
configfile = './dmpipe_example/config/master_dSphs.yaml'

We have to 'preconfigure' the pipeline, because we need to build up the list of targets so that we correctly set up the later stages of the pipeline.

In [5]:
pipe.preconfigure(configfile)

return code  1


In [6]:
!DMSKY_PATH=dmpipe_example/dmsky_data

We then tell the pipeline to update the arguments for all of the steps that comprise it.

In [7]:
pipe.update_args(dict(config=configfile))

FileNotFoundError: [Errno 2] No such file or directory: './dSphs/target_list.yaml'

Now we look around a bit, we can drill down into the links that make up the pipeline.
In this case we have a pipeline that consists of a couple of preparation jobs,
then 5 sub-pipelines ('data', 'sim_xxx', 'random').
The sub-pipeline analyze the real data, and some different simulation cases.
The 'random' sub-pipeline analyzes random sky directions.

At this point the pipeline is fully configured, let's look around.

In [ ]:
pipe.linknames

In [ ]:
pipe['data']

In [ ]:
pipe['data'].linknames

We can print the status for the various links.
Using recurse=True will also drill down to the status of the links in the sub pipelines.

In [ ]:
pipe.print_status()

In [ ]:
pipe.print_status(recurse=True)

We can access a particular Link in the pipeline.  
We can ask:
  what the default options for the link are ('\_options')
  what the current set of options are ('args')

In [ ]:
pipe['data']['analyze-roi']

In [ ]:
pipe['data']['analyze-roi']._options

In [ ]:
pipe['data']['analyze-roi'].args

We can ask what jobs get run by this link.   In this paritcular case the link
runs 1 job that in turn dispatches several other jobs to the batch farm.

In [ ]:
pipe['data']['analyze-roi'].jobs

Here we are talking to the link that represents any one of the disptached jobs.
The command_template() function tells us how we would run this job from the UNIX command line.

In [ ]:
pipe['data']['analyze-roi'].scatter_link

In [ ]:
pipe['data']['analyze-roi'].scatter_link.command_template()

Here are ask what jobs will be dispatched.  Note that there are two jobs with slightly different names.

In [ ]:
pipe['data']['analyze-roi'].scatter_link.jobs

Here are are asking for information about the first of those two jobs.  In particular, the specific options used for this instance of the job.  You can merge the job_config with the command_template to get the exact syntax for the instance of this command.

In [ ]:
pipe['data']['analyze-roi'].scatter_link.jobs['draco@dSphs.data.analyze-roi']

In [ ]:
pipe['data']['analyze-roi'].scatter_link.jobs['draco@dSphs.data.analyze-roi'].job_config

Here we run a single link.  
We use 'run_with_log()' so that the log file will be in the correct place and the pipeline will know the job
has completed.             

In [ ]:
pipe['spec-table'].run_with_log()

In [ ]:
pipe.print_status()

Here we run the 'data' sub-pipeline.

In [ ]:
pipe['data'].run(resubmit_failed=True)

In [ ]:
pipe.print_status(recurse=True)

In [ ]:
pipe['data']['convert-castro'].scatter_link.jobs['draco:ack2016_point:lgauss@dSphs.data.convert-castro'].job_config

In [ ]:
pipe.run(resubmit_failed=True)